In [1]:
import sys
sys.path.append('..')

In [2]:
import torch

if torch.cuda.is_available():
    device = 'cuda'
if torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

In [3]:
from src.dataset.footprint2pressure import Footprint2Pressure

dataset = Footprint2Pressure(
    device = device,
    footprint_wrap_folder = '..data/processed/footprint-wrap',
    pedar_dynamic_path = '..data/processed/pedar_dynamic.pkl',
    l_mask_path = '..data/processed/left_foot_mask.png',
)

In [28]:
pedar_arr = dataset.pedar_dynamic.to_numpy() / dataset.sense_range
pedar_arr_t = torch.Tensor(pedar_arr).to(device)
pedar_arr_t.shape

torch.Size([300, 198])

## Naive model: average array

In [29]:
pedar_arr_mean = dataset.pedar_dynamic.mean().to_numpy() / dataset.sense_range
pedar_arr_mean_t = torch.Tensor(pedar_arr_mean).expand(pedar_arr_t.shape).to(device)
pedar_arr_mean_t.shape

torch.Size([300, 198])

In [33]:
from torch.nn import L1Loss, MSELoss

{
    'MAE': L1Loss()(pedar_arr_t, pedar_arr_mean_t),
    'MSE': MSELoss()(pedar_arr_t, pedar_arr_mean_t),
}

{'MAE': tensor(0.0299, device='mps:0'), 'MSE': tensor(0.0018, device='mps:0')}

## Naive model: average value

In [45]:
pedar_mean = pedar_arr_mean.mean()
pedar_mean_t = torch.Tensor([pedar_mean]).expand(pedar_arr_t.shape).to(device)
pedar_mean_t.shape

torch.Size([300, 198])

In [46]:
{
    'MAE': L1Loss()(pedar_arr_t, pedar_mean_t),
    'MSE': MSELoss()(pedar_arr_t, pedar_mean_t),
}

{'MAE': tensor(0.0462, device='mps:0'), 'MSE': tensor(0.0037, device='mps:0')}